# Requiments

Mount Google Drive before run, otherwise the parser won't work.

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

# Parser

Every time we open colab, it loads the default enviroment, in which is not installed xlsxwriter. So we need to install the xlsxwriter package every time.

In [1]:
pip install xlsxwriter # because it's not a standard package inside Colab

     |████████████████████████████████| 141 kB 177 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import xlsxwriter
import re
from collections import namedtuple
from google.colab import files
import io

ModuleNotFoundError: No module named 'numpy'

In [0]:
#input_file = "out.txt"
input_file = "drive/My Drive/Magistrale/Corsi/IALAB/Progetto/19-20/ASP/output_clingo/out_completed_utf-8.txt"

Calendar = namedtuple("Calendar",["week","day","hour","id","lecture","professor"]) 
  
with open(input_file,"r") as file:
    lines = file.read()
    
    ## Parser specification
    regex_predicate = re.compile("calendar\((\d+),(\d+),(\d+),(\d+),\"([\w\s:-]+)\",\"([\w\s:-]*)\"\)") # senza lecture
    regex_top_level = re.compile("(Answer: [\d]+)\n(.+)")
    
    # Parse 2 hardcoded nested levels: 
    # lvl 1) multiple answer-set
    parsed_content = {}
    for answer_set_match in re.finditer(regex_top_level,lines):
        answer_set_name = answer_set_match.groups()[0]
        answer_set =  answer_set_match.groups()[1]
        # lvl 2) predicates
        predicates = [Calendar._make(predicate_match.groups()) for predicate_match in re.finditer(regex_predicate,answer_set)]
        
        # build dictionary of parsed answer-sets (normalizing the key value)
        parsed_content[answer_set_name.replace(": ","_")] = predicates

# TODO: put in other cell.
def format_excel(file_name):
    
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    df.to_excel(writer, sheet_name="Sheet1")

    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']

    # Format 1. Light red fill with dark red text.
    format1 = workbook.add_format({'bg_color': '#FFC7CE',
                                  'font_color': '#9C0006'})

    # Format 2. Green fill with dark green text.
    format2 = workbook.add_format({'bg_color': '#C6EFCE',
                                  'font_color': '#006100'})
    
    # Format 3. Yellow fill with brown text.
    format3 = workbook.add_format({'bg_color': '#FFEB9B',
                                  'font_color': '#9D744F'})
    
    # Format 4. Orange fill with light purple text.
    format4 = workbook.add_format({'bg_color': '#FFCC99',
                                  'font_color': '#766882'})

    worksheet.conditional_format('F2:F359', {'type': 'text',
                                            'criteria': 'containing',
                                            'value': 'Progettazione e sviluppo di applicazioni web su dispositivi mobile I',
                                            'format': format1})
    
    worksheet.conditional_format('F2:F359', {'type': 'text',
                                            'criteria': 'containing',
                                            'value': 'Progettazione e sviluppo di applicazioni web su dispositivi mobile II',
                                            'format': format2})

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    print("Formatting of file {} completed.".format(file_name))

In [0]:
parsed_content

In [0]:
df = pd.DataFrame(parsed_content['Answer_1'])

df = df.astype({'week': 'int64', 'day': 'int64', 'hour': 'int64'})

In [0]:
df = df.set_index(['week','day','hour']).drop('id',axis=1)

df = df.sort_index()

df = df.replace({'day':{'1':'Lun (1)','2':'Mar (2)','3':'Mer (3)',
                   '4':'Gio (4)','5':'Ven (5)','6':'Sab (6)'}})


In [0]:
df['Lecture'] = df['lecture'] + ": " + df['professor']
df = df.drop(['lecture','professor'], axis=1)

In [0]:
df

##Build TimeTable (TODO)

In [0]:
timetable = df.unstack('hour')

timetable = timetable.apply(lambda x: pd.Series(x.dropna().values), axis=1).fillna('')
#timetable = timetable.apply(lambda x: pd.Series(x.dropna().values), axis=1).fillna('')


timetable = timetable.set_axis([1,2,3,4,5,6,7,8], axis = 1)
timetable = timetable.rename_axis("hour", axis=1)
timetable

In [0]:
timetable.to_excel("timetable.xlsx")

## Export and unmount Google Drive
**Manually coping** the Excel sheet containing the result inside my Google Drive folder

In [0]:
!cp timetable.xlsx "drive/My Drive/Magistrale/Corsi/IALAB/Progetto/19-20/ASP/output_excel"

Finally unmount Google Drive

In [0]:
drive.flush_and_unmount()